![eodag logo](https://eodag.readthedocs.io/en/latest/_static/eodag_bycs.png)

# eodag-labextension: user manual

The `eodag-labextension` JupyterLab plugin brings satellite imagery search ability. With that extension you can search products, see results and check if results corresponds your needs. And finally generate Python code allowing to access to those products in the notebook.

That tool is based on [eodag](https://github.com/CS-SI/eodag). See the [Python API User Guide](https://eodag.readthedocs.io/en/stable/api_user_guide.html) for a complete overview of eodag features.

## Configuration

For some providers we don’t need to set up credentials to search for products. But if you wish to download them, you should set the credentials beforehand, using these two environment variables for instance.

In [ ]:
# Example for PEPS
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__USERNAME"] = "PLEASE_CHANGE_ME"
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__PASSWORD"] = "PLEASE_CHANGE_ME"

Credendials can be set dynamically using [environment variables](https://eodag.readthedocs.io/en/stable/getting_started_guide/configure.html#environment-variable-configuration) or also stored in the [configuration file](https://eodag.readthedocs.io/en/stable/getting_started_guide/configure.html#yaml-configuration-file) (recommanded). This file will be located at `$HOME/.config/eodag/eodag.yml`.

You can always choose to configure eodag using the configuration file or dynamically within your code, as shown in the [Configuration](https://eodag.readthedocs.io/en/stable/notebooks/api_user_guide/3_configuration.html) page of eodag's documentation.

Configuration also includes providers priority, download destination (`outputs_prefix`), extraction, ...

## Search

![extension logo](https://raw.githubusercontent.com/CS-SI/eodag-labextension/develop/notebooks/images/eodag_labext_icon.png) Activate the plugin tab on the left of the JupuytuerLab interface by clicking on that icon.

![extension form](https://raw.githubusercontent.com/CS-SI/eodag-labextension/develop/notebooks/images/eodag_labext_form.png)

With displayed search form, you can enter search extent and following search criteria:

* **Product type**: the searched product type. For each entry of the drop-down list, a tooltip is displayed at hovering time with corresponding description. 
* **Start date**: minimal date of the search temporal window.
* **End date**: maximal date of the search temporal window.
* **Max cloud cover**: maximum cloud cover allowed in search results in percent.
* **Additional parameters**: used to enter key-value pairs criteria for the request.

You can draw multiple extents, or use none. Each extent can be a rectangle or a free polygon.
Product type is mandatory. Other criteria are optional.

Once search criteria are filled out, click on the "Search" button to proceed to next step. At the end of the search, a popup opens and displays results.

## Results overview

![extension popup](https://raw.githubusercontent.com/CS-SI/eodag-labextension/develop/notebooks/images/eodag_labext_popup.png)

The results overview popup is composed of 3 parts:

* a map showing products extent,
* a table listing products,
* a pane containing metadata for currently selected product.

The results table allows you to access product metadata by clicking on the desired product line. The magnifying glass button allows you to zoom in on the product's in the map view. By scrolling down in the list of results, the search is automatically restarted to retrieve the following results.

In the metadata view, clicking on the thumbnail displays it in native resolution. Clicking it again reduces its size again.

## Apply to the Jupyter notebook

If the search result is correct, clicking on the "`Apply`" button will insert the Python eodag code in a new cell of the currently open notebook. The popup is automatically closed. From there, it is possible to work in the notebook on the search results by executing the eodag search.

Here is an example of generated code:

In [ ]:
from eodag import EODataAccessGateway, setup_logging

setup_logging(1) # 0: nothing, 1: only progress bars, 2: INFO, 3: DEBUG

dag = EODataAccessGateway()
geometry = "POLYGON ((0.550136 43.005451, 0.550136 44.151469, 2.572104 44.151469, 2.572104 43.005451, 0.550136 43.005451))"
search_results, total_count = dag.search(
  productType="S2_MSI_L1C",
  geom=geometry,
  start="2021-08-01",
  end="2021-08-11",
  cloudCover=17,
)

You may want to enforce usage of a particular provider. To do so, use [set_preferred_provider()](https://eodag.readthedocs.io/en/stablesearch/notebooks/api_user_guide/3_configuration.html?#Set-a-provider's-priority) in  your search code after having instanciated [EODataAccessGateway](https://eodag.readthedocs.io/en/stable/api_reference/core.html#eodag.api.core.EODataAccessGateway):

In [ ]:
dag = EODataAccessGateway()
dag.set_preferred_provider("theia")

## Using results

The obtained `SearchResult` will contain several `EOProduct` objects. See [SearchResult and EOProduct](https://eodag.readthedocs.io/en/stable/notebooks/api_user_guide/4_search.html#SearchResult-and-EOProduct) in the documentation for more information.

Here are some examples about how to use search results into a notebook:

In [ ]:
from pprint import pprint

# Display results list
pprint(search_results)

# Display products access paths
pprint([p.location for p in search_results])

### Extract products extent

In [ ]:
from shapely.geometry import GeometryCollection

features = GeometryCollection(
    [product.geometry for product in search_results]
)
features

### Display products extent on a slippy map

In [ ]:
from folium import Map, GeoJson, Figure

ext = features.bounds
bounds = [[ext[1], ext[0]], [ext[3], ext[2]]]
m = Map(tiles="Stamen Terrain", control_scale=True,)
GeoJson(search_results).add_to(m)
m.fit_bounds(bounds)
Figure(width=500, height=300).add_child(m)

## Downloading products

See [Download EO products](https://eodag.readthedocs.io/en/stable/notebooks/api_user_guide/7_download.html#Download-EO-products) in the documentation.

To download all products from the search request into a sub-directory called `downloaded`, run:

In [ ]:
dag.download_all(search_results, outputs_prefix="downloaded")